In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.uoguelph.ca/engineering/mechanical-engineering-restricted-electives-20202021"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for driver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
containers = page_soup.find("div", {"class": "field-item even"}).findAll("a")

In [8]:
link_texts = [container.text for container in containers if bool(re.match("ENGG\*[0-9]{4}", container.text))]
len(link_texts)

26

In [17]:
link_texts

['ENGG*2410',
 'ENGG*3070 ',
 'ENGG*3080',
 'ENGG*3150',
 'ENGG*3390',
 'ENGG*3450',
 'ENGG*3640',
 'ENGG*3700',
 'ENGG*4230',
 'ENGG*4460',
 'ENGG*4470',
 'ENGG*3120',
 'ENGG*3170',
 'ENGG*3250',
 'ENGG*3490',
 'ENGG*4050',
 'ENGG*4490',
 'ENGG*4430',
 'ENGG*4440',
 'ENGG*4510',
 'ENGG*4660',
 'ENGG*4030',
 'ENGG*4220',
 'ENGG*4400',
 'ENGG*4480',
 'ENGG*4580']

# 2. Scrape courses

In [12]:
driver.find_element_by_id("becookiebarbuttonid").click()

In [13]:
driver.find_element_by_link_text(link_texts[0]).click() #go to page listing all engg courses. Scrape from there

In [14]:
page_soup = soup(driver.page_source, 'lxml')

In [15]:
containers = page_soup.findAll("div", {"class": "courseblock"})

Turns out all courses will have to be found and scraped in one webpage listing all engg courses. So that's what the script below will do, using the link texts collected earlier to find the target courses

In [18]:
counter = 0
course_names = []
course_codes = []
course_descs = []

for link_text in link_texts:
    for container in containers:
        if container.find("span", {"class": "text detail-code margin--small text--semibold text--big"}).text.strip() == link_text.strip():
            course_codes.append(container.find("span", {"class": "text detail-code margin--small text--semibold text--big"}).text.strip())
            course_names.append(container.find("span", {"class": "text detail-title margin--small text--semibold text--big"}).text.strip())
            course_descs.append(container.find("p", {"class": "courseblockextra noindent"}).text)
            
            counter += 1
            continue
            
print("scraped {} courses".format(counter))

scraped 26 courses


# 3. Inspect and write to CSV

In [19]:
course_codes

['ENGG*2410',
 'ENGG*3070',
 'ENGG*3080',
 'ENGG*3150',
 'ENGG*3390',
 'ENGG*3450',
 'ENGG*3640',
 'ENGG*3700',
 'ENGG*4230',
 'ENGG*4460',
 'ENGG*4470',
 'ENGG*3120',
 'ENGG*3170',
 'ENGG*3250',
 'ENGG*3490',
 'ENGG*4050',
 'ENGG*4490',
 'ENGG*4430',
 'ENGG*4440',
 'ENGG*4510',
 'ENGG*4660',
 'ENGG*4030',
 'ENGG*4220',
 'ENGG*4400',
 'ENGG*4480',
 'ENGG*4580']

In [20]:
course_names

['Digital Systems Design Using  Descriptive Languages',
 'Integrated Manufacturing Systems',
 'Energy Resources and Technologies',
 'Engineering Biomechanics',
 'Signal Processing',
 'Electronic Devices',
 'Microcomputer Interfacing',
 'Optimization for Engineers',
 'Energy Conversion',
 'Robotic Systems',
 'Finite Element Analysis',
 'Computer Aided Design and Manufacturing',
 'Biomaterials',
 'Energy Management and Utilization',
 'Introduction to Mechatronic Systems  Design',
 'Quality Control',
 'Sampled Data Control Design',
 'Neuro-Fuzzy and Soft Computing Systems',
 'Computational Fluid Dynamics',
 'Assessment and Management of Risk',
 'Medical Image Processing',
 'Manufacturing System Design',
 'Interdisciplinary Mechanical  Engineering Design',
 'Biomechanical Engineering Design',
 'Advanced Mechatronic Systems Design',
 'Sustainable Energy Systems Design']

In [21]:
course_descs

['Review of Boolean algebra and truth tables,  Karnaugh maps. Design, synthesis and realization  of combinational circuits. Design, synthesis and  realization of sequential circuits. VHDL:  structural modeling, data flow modeling,  synchronous & asynchronous behavior descriptions,  algorithmic modeling. Designing with PLDs.  Digital design with SM charts. Designing with  PGAs and complex programmable logical devices.  Hardware testing and design for testability.  Hierarchy in large designs. The course will  primarily be concerned with the design of  multi-input, multi-output digital controllers  which provide the central control signals that  orchestrate the collection of hardware devices  (from SSI to VLSI) found in a digital system. An  introduction to FPGA-based, as well as  microprocessor-based digital systems design will  be given. Design examples will include systems  such as UART, microcontroller CPU, ALU and data  acquisition system.',
 'Common production machines and manufactu

In [22]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
})

df

,Course Number,Course Name,Course Description
0,ENGG*2410,Digital Systems Design Using Descriptive Lang...,"Review of Boolean algebra and truth tables, K..."
1,ENGG*3070,Integrated Manufacturing Systems,Common production machines and manufacturing ...
2,ENGG*3080,Energy Resources and Technologies,The challenges of changing the global energy ...
3,ENGG*3150,Engineering Biomechanics,The following topics related to biomechanics a...
4,ENGG*3390,Signal Processing,This course will establish the fundamental an...
5,ENGG*3450,Electronic Devices,This course explores the theory and principles...
6,ENGG*3640,Microcomputer Interfacing,This course focuses on the subject of interfac...
7,ENGG*3700,Optimization for Engineers,This course serves as an introduction to opti...
8,ENGG*4230,Energy Conversion,The course introduces the technical criteria f...
9,ENGG*4460,Robotic Systems,This course covers robot technology fundamenta...


In [23]:
df.to_csv('UGuelph_MechEng_Technical_Electives_Courses.csv', index = False)

In [24]:
driver.quit()